In [1]:
import requests
import pandas as pd
import json
from keys import claves as keys
from datetime import date, timedelta
import psycopg2
key = keys.ver(303)

In [2]:
url="https://api.weatherbit.io/v2.0/history/daily?lat={0}&lon={1}&start_date={2}&end_date={3}&key={4}"


latitude = 40.776676
longitude = -73.971321

year = 2022
month = 1
day = 1

In [3]:
start_date = date(year,month,day) # 2023-04-04
end_date = start_date + timedelta(1) # Request required furter date

full_url = url.format(latitude,longitude,start_date,end_date,key)

In [4]:
response = requests.get(full_url)
print(response.text)
if response.status_code == 200:
    print("OK")

{"city_id":"5125771","city_name":"Manhattan","country_code":"US","data":[{"clouds":100,"datetime":"2022-01-01","dewpt":10.4,"dhi":27,"dni":229,"ghi":99,"max_dhi":91,"max_dni":772,"max_ghi":426,"max_temp":13.3,"max_temp_ts":1641074400,"max_uv":0.7,"max_wind_dir":157,"max_wind_spd":4.5,"max_wind_spd_ts":1641063600,"min_temp":10,"min_temp_ts":1641031200,"precip":38.5,"precip_gpm":38.5,"pres":1003,"revision_status":"final","rh":92.6,"slp":1008,"snow":0,"snow_depth":0,"solar_rad":10,"t_dhi":637,"t_dni":5502,"t_ghi":2368,"t_solar_rad":237,"temp":11.6,"ts":1641013200,"wind_dir":157,"wind_gust_spd":3.9,"wind_spd":2.1}],"lat":40.776676,"lon":-73.971321,"sources":["720553-99999","725053-94728","USW00094728","imerg","merra2","era5","modis"],"state_code":"NY","station_id":"725053-94728","timezone":"America/New_York"}

OK


In [5]:
json_data = json.loads(response.text, parse_constant=lambda x: x.replace("'", '"'))
df = pd.json_normalize(json_data, record_path =['data'])

In [6]:
def conection():
    conn = psycopg2.connect(
        host="data-engineer-cluster.cyhh5bfevlmn.us-east-1.redshift.amazonaws.com",
        database="data-engineer-database",
        user= keys.ver(101),
        password= keys.ver(202),
        port="5439")
    
    return conn
conn = conection()
cursor = conn.cursor()

In [7]:
a = df.dtypes

In [8]:
for column in df.columns:
    print(column, df[column].dtype)

clouds int64
datetime object
dewpt float64
dhi int64
dni int64
ghi int64
max_dhi int64
max_dni int64
max_ghi int64
max_temp float64
max_temp_ts int64
max_uv float64
max_wind_dir int64
max_wind_spd float64
max_wind_spd_ts int64
min_temp int64
min_temp_ts int64
precip float64
precip_gpm float64
pres int64
revision_status object
rh float64
slp int64
snow int64
snow_depth int64
solar_rad int64
t_dhi int64
t_dni int64
t_ghi int64
t_solar_rad int64
temp float64
ts int64
wind_dir int64
wind_gust_spd float64
wind_spd float64


In [9]:
dic_pdtosql = {
    'object': 'VARCHAR',
    'int64': 'INTEGER',
    'float64': 'FLOAT',
    'bool': 'BIT',
    'datetime64': 'DATETIME',
    'timedelta64': 'INTERVAL',
    'category': 'VARCHAR'
}

In [10]:
table_name = "daily_weather"
columns = []

In [11]:
for column in df.columns:
    data_type = dic_pdtosql[str(df[column].dtype)]
    columns_sql = '{} {}'.format(column, data_type)
    columns.append(columns_sql)

In [14]:
for i in columns:
    print(i)

clouds INTEGER
datetime VARCHAR
dewpt FLOAT
dhi INTEGER
dni INTEGER
ghi INTEGER
max_dhi INTEGER
max_dni INTEGER
max_ghi INTEGER
max_temp FLOAT
max_temp_ts INTEGER
max_uv FLOAT
max_wind_dir INTEGER
max_wind_spd FLOAT
max_wind_spd_ts INTEGER
min_temp INTEGER
min_temp_ts INTEGER
precip FLOAT
precip_gpm FLOAT
pres INTEGER
revision_status VARCHAR
rh FLOAT
slp INTEGER
snow INTEGER
snow_depth INTEGER
solar_rad INTEGER
t_dhi INTEGER
t_dni INTEGER
t_ghi INTEGER
t_solar_rad INTEGER
temp FLOAT
ts INTEGER
wind_dir INTEGER
wind_gust_spd FLOAT
wind_spd FLOAT


In [15]:
make_table = "CREATE TABLE {} ({})".format(table_name, ", ".join(columns))

CREATE TABLE daily_weather (clouds INTEGER, datetime VARCHAR, dewpt FLOAT, dhi INTEGER, dni INTEGER, ghi INTEGER, max_dhi INTEGER, max_dni INTEGER, max_ghi INTEGER, max_temp FLOAT, max_temp_ts INTEGER, max_uv FLOAT, max_wind_dir INTEGER, max_wind_spd FLOAT, max_wind_spd_ts INTEGER, min_temp INTEGER, min_temp_ts INTEGER, precip FLOAT, precip_gpm FLOAT, pres INTEGER, revision_status VARCHAR, rh FLOAT, slp INTEGER, snow INTEGER, snow_depth INTEGER, solar_rad INTEGER, t_dhi INTEGER, t_dni INTEGER, t_ghi INTEGER, t_solar_rad INTEGER, temp FLOAT, ts INTEGER, wind_dir INTEGER, wind_gust_spd FLOAT, wind_spd FLOAT)


In [16]:
with cursor:
    cursor.execute(make_table)